In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 7


In [12]:
import warnings

warnings.filterwarnings("ignore")
from pprint import pprint

from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language


In [13]:
loader = GenericLoader.from_filesystem(
    "./DotNetFramework/",
    glob="*",
    suffixes=[".cs"],
    parser=LanguageParser(language=Language.CSHARP, parser_threshold=1000),    
)
docs = loader.load()

In [14]:
docs[0].metadata['source']

'DotNetFramework/file1.cs'

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

code_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=1000, chunk_overlap=0
)
documents = code_splitter.split_documents(docs)



In [16]:
documents[0].page_content

'using System;\nusing System.Web.UI;\nusing System.Web.UI.WebControls;\n\nnamespace WebFormsApp\n{\n    public class CustomWebControl : WebControl\n    {\n        protected override void RenderContents(HtmlTextWriter writer)\n        {\n            writer.Write("Hello, this is a custom WebControl for .NET Framework WebForms.");\n        }\n    }\n\n    public partial class WebForm1 : Page\n    {\n        protected void Page_Load(object sender, EventArgs e)\n        {\n            CustomWebControl myControl = new CustomWebControl();\n            myControl.ID = "MyCustomControl";\n            this.form1.Controls.Add(myControl);\n        }\n    }\n}'

In [17]:
## FAISS Vector Database
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [18]:
# from langchain_community.llms import Ollama
# llm=Ollama(model="llama2")

from langchain_openai import ChatOpenAI
# llm=ChatOpenAI(model="gpt-3.5-turbo")
llm=ChatOpenAI(model="gpt-4-turbo")


In [19]:
# ## Design ChatPrompt Template
# from langchain_core.prompts import ChatPromptTemplate

# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. please rewrite the piece of code with .NET 5/6+ or ASP.NET Core."),
#         ("user","old code:{oldcode}")
#     ]
# )



## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Hi ChatGPT, we are going to rewrite some old C# code which is based on .NET Framework. please rewrite the piece of code with .NET 6+ or ASP.NET Core. 
In addition, basedon the context provided, please help to identify.Net Framework features replaced. 
Please use xml tag <description></description> to contain all your description; and use xml tag <NewCode><file name="xxxx"></file><file name="yyyy"></file>...</NewCode> to wrap source code files generated.
<context>
{context}
</context>
Old Code: {input}"""
)


In [ ]:
## Chain Introduction
## Create Stuff Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [20]:
"""A retriever is an interface that returns documents given an unstructured query. 
It is more general than a vector store. A retriever does not need to be able to store documents,
only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, 
but there are other types of retrievers as well.
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f84718bebd0>)

In [21]:
from langchain.chains import create_retrieval_chain

retriever_chain  = create_retrieval_chain(retriever, document_chain)

In [22]:
response=retriever_chain.invoke({"input": documents[0].page_content})

In [23]:
response['answer']

'<description>\nThe provided code snippet is based on ASP.NET Web Forms, a framework part of the older .NET Framework used for building dynamic web applications. In .NET 5/6 and later versions, ASP.NET Core is used instead, which does not support Web Forms. Thus, the Web Forms code needs to be rewritten using ASP.NET Core MVC or Razor Pages, which are the recommended approaches for web UI in .NET Core and beyond.\n\nKey changes include:\n1. The `WebControl` class and `HtmlTextWriter` are specific to Web Forms and have no direct equivalents in ASP.NET Core. Instead, components are typically built using Razor views or Tag Helpers.\n2. Event-driven model of Web Forms (like `Page_Load`) is replaced by the MVC (Model-View-Controller) or Razor Pages pattern in ASP.NET Core, which separates concerns and improves testability and modularity.\n\nFor demonstration, I\'ll rewrite the example using ASP.NET Core MVC with a simple view component that serves a similar purpose as the original `CustomWe

In [24]:
import re
from IPython.display import display, Markdown

# Your input string
input_string = response['answer']



code_match = re.search(r'<NewCode>\n(.*?)\n</NewCode>', input_string, re.DOTALL)
description_match = re.search(r'<description>\n(.*?)\n</description>', input_string, re.DOTALL)

# Extracted code and description
code = code_match.group(1) if code_match else None
description = description_match.group(1) if description_match else None

print(code)


display(Markdown(description))



<file name="CustomWebControlViewComponent.cs">
using Microsoft.AspNetCore.Mvc;

namespace AspNetCoreApp
{
    public class CustomWebControlViewComponent : ViewComponent
    {
        public IViewComponentResult Invoke()
        {
            return Content("Hello, this is a custom WebControl for ASP.NET Core.");
        }
    }
}
</file>
<file name="HomeController.cs">
using Microsoft.AspNetCore.Mvc;

namespace AspNetCoreApp.Controllers
{
    public class HomeController : Controller
    {
        public IActionResult Index()
        {
            return View();
        }
    }
}
</file>
<file name="Index.cshtml">
@{
    ViewData["Title"] = "Home Page";
}

<div>
    @Component.InvokeAsync("CustomWebControl")
</div>
</file>
<file name="Startup.cs">
using Microsoft.AspNetCore.Builder;
using Microsoft.AspNetCore.Hosting;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Hosting;

namespace AspNetCoreApp
{
    public class Startup
    {
        public void Configure

The provided code snippet is based on ASP.NET Web Forms, a framework part of the older .NET Framework used for building dynamic web applications. In .NET 5/6 and later versions, ASP.NET Core is used instead, which does not support Web Forms. Thus, the Web Forms code needs to be rewritten using ASP.NET Core MVC or Razor Pages, which are the recommended approaches for web UI in .NET Core and beyond.

Key changes include:
1. The `WebControl` class and `HtmlTextWriter` are specific to Web Forms and have no direct equivalents in ASP.NET Core. Instead, components are typically built using Razor views or Tag Helpers.
2. Event-driven model of Web Forms (like `Page_Load`) is replaced by the MVC (Model-View-Controller) or Razor Pages pattern in ASP.NET Core, which separates concerns and improves testability and modularity.

For demonstration, I'll rewrite the example using ASP.NET Core MVC with a simple view component that serves a similar purpose as the original `CustomWebControl`.

In [25]:
pattern = re.compile(r'<file name="(.+?)">(.*?)</file>', re.DOTALL)

# Finding all matches
matches = pattern.findall(code)

# Loop through matches and write to files
for file_name, file_content in matches:
    # Strip leading and trailing whitespaces from the content
    file_content = file_content.strip()
    # Open the file in write mode and write the content
    with open("./DotNet/"+file_name, 'w') as file:
        file.write(file_content)
    print(f"File saved: {file_name}")

File saved: CustomWebControlViewComponent.cs
File saved: HomeController.cs
File saved: Index.cshtml
File saved: Startup.cs
